# Chapter 9: CycleGAN

# Imports

In [240]:
import torch
import torch.nn as nn
from torchvision.datasets import VisionDataset, ImageFolder
from torchvision.datasets.utils import check_integrity, download_and_extract_archive
from torchvision.transforms import Compose, ToTensor, Resize
from torch.utils.data import DataLoader
import os
import matplotlib.pyplot as plt

# Loading data

PyTorch or torchvision does not come with the `apple2orange` dataset. Therefore we implement a full dataset which retrieves the data.

In [214]:
class Apple2Orange(VisionDataset):
    url = "https://people.eecs.berkeley.edu/~taesung_park/CycleGAN/datasets/apple2orange.zip"
    base_folder = "apple2orange"
    filename = "apple2orange.zip"
    zip_md5 = "5b58c340256288622a835d6f3b6198ae"
    
    def __init__(self, root, train=True, transform=None, target_transform=None,
             download=False):
        super().__init__(root, transform=transform,
                            target_transform=target_transform)
        self.train = train
        self.train_folder = os.path.join(self.root, self.base_folder, "train")
        self.test_folder = os.path.join(self.root, self.base_folder, "test")

        if download:
            self.download()
            
        if train:
            root = self.train_folder
        else:
            root = self.test_folder
            
        self.dataset = ImageFolder(root, transform, target_transform)
        
    def __getitem__(self, index):
        return self.dataset[index]
    
    def __len__(self):
        return len(self.dataset)
            
    def _check_integrity(self):
        fpath = os.path.join(self.root, self.filename)
        
        if check_integrity(fpath, self.zip_md5) and os.path.exists(os.path.join(self.root, self.base_folder)):
            return True
        else:
            return False
        
            
    def download(self):
        if self._check_integrity():
            print('Files already downloaded and verified')
            return
        
        #download_and_extract_archive(self.url, self.root, filename=self.filename, md5=self.zip_md5)
        
        train_path = os.path.join(self.root, self.base_folder, "train")
        test_path = os.path.join(self.root, self.base_folder, "test")
        source_train_a = os.path.join(self.root, self.base_folder, "trainA")
        source_train_b = os.path.join(self.root, self.base_folder, "trainB")
        source_test_a = os.path.join(self.root, self.base_folder, "testA")
        source_test_b = os.path.join(self.root, self.base_folder, "testB")
        
        os.mkdir(train_path)
        os.mkdir(test_path)
        
        os.renames(source_train_a, os.path.join(train_path, "apple"))
        os.renames(source_train_b, os.path.join(train_path, "orange"))
        os.renames(source_test_a, os.path.join(test_path, "apple"))
        os.renames(source_test_b, os.path.join(test_path, "orange"))


In [241]:
t = Compose([Resize(128), ToTensor()])
train_set = Apple2Orange('~/pytorch', train=True, download=True, transform=t)
test_set = Apple2Orange('~/pytorch', train=False, download=True, transform=t)
train_loader = DataLoader(train_set, batch_size=8, shuffle=True, drop_last=True)

Files already downloaded and verified
Files already downloaded and verified


# Generator

In [172]:
class Generator(nn.Module):
    class TransSkip(nn.Module):
        def __init__(self, in_channel, out_channel, kernel_size):
            super().__init__()
            self.trans1 = nn.Sequential(
                nn.UpsamplingBilinear2d(scale_factor=2),
                nn.Conv2d(in_channel, out_channel, kernel_size, stride=1, padding=int(kernel_size/2)),
                nn.LeakyReLU(negative_slope=0.2),
                nn.InstanceNorm2d(out_channel/2))
            
        def forward(self, x, skip):
            trans = self.trans1(x)
            x = torch.cat([trans,  skip], dim=1)
            
            return x

    def __init__(self):
        super().__init__()
        self.conv1 = self.conv_layer(3, 32, 3)
        self.conv2 = self.conv_layer(32, 64, 3)
        self.conv3 = self.conv_layer(64, 128, 3)
        self.conv4 = self.conv_layer(128, 256, 3)
        
        self.trans1 = self.TransSkip(256, 128, 3)
        self.trans2 = self.TransSkip(256, 64, 3)
        self.trans3 = self.TransSkip(128, 32, 3)
        # Check if using skip from first image improves result
        # self.trans4 = self.TransSkip(64, 64, 3)
        # self.conv5 = nn.Sequential(
        #     nn.Conv2d(67, 3, 3, stride=1, padding=1),
        #     nn.Tanh())
        self.trans4 = nn.UpsamplingBilinear2d(scale_factor=2)
        self.conv5 = nn.Sequential(
            nn.Conv2d(64, 3, 3, stride=1, padding=1),
            nn.Tanh())
    
    def forward(self, x):
        skip_0 = x
        skip_1 = x = self.conv1(x)
        skip_2 = x = self.conv2(x)
        skip_3 = x = self.conv3(x)
        x = self.conv4(x)
        x = self.trans1(x, skip_3)
        x = self.trans2(x, skip_2)
        x = self.trans3(x, skip_1)
        # x = self.trans4(x, skip_0)
        x = self.trans4(x)
        x = self.conv5(x)
        
        return x
    
    def conv_layer(self, in_channels, out_channels, kernel_size):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size, stride=2, padding=1),
            nn.LeakyReLU(),
            nn.InstanceNorm2d(out_channels))

In [173]:
generator = Generator()

In [174]:
out = generator(torch.randn(8, 3, 128, 128))

In [175]:
out.shape

torch.Size([8, 3, 128, 128])

In [176]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = self.conv_layer(3, 64, 3)
        self.conv2 = self.conv_layer(64, 128, 3)
        self.conv3 = self.conv_layer(128, 256, 3)
        self.conv4 = self.conv_layer(256, 512, 3)
        self.conv5 = nn.Conv2d(512, 1, 4, stride=1, padding=2)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        
        return x
    
    def conv_layer(self, in_channels, out_channels, kernel_size):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size, stride=2, padding=int(kernel_size/2)),
            nn.LeakyReLU(negative_slope=0.2))

In [177]:
discriminator = Discriminator()

In [178]:
out = discriminator(torch.randn(8, 3, 128, 128))

In [179]:
out.shape

torch.Size([8, 1, 9, 9])